In [3]:
from abc import abstractmethod

class LLM:
    
    @abstractmethod
    def generate(self, system_prompt: str, user_prompt: str) -> str:
        """
        Generate an answer to prompt passed as a parameter
        
        Args:
            system_prompt: The prompt that defines the LLM
            user_prompt: The prompt to answer
        
        Returns:
            The output for that prompt
        """

class LLama3(LLM):

    def __init__(self):
        from dotenv import load_dotenv
        from groq import Groq
        import os

        load_dotenv()
        self.client = Groq(api_key=os.environ.get("GROQ_API_KEY"))
    
    def generate(self, system_prompt: str, user_prompt: str) -> str:
        chat_completion = self.client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": system_prompt
                },
                {
                    "role": "user",
                    "content": user_prompt
                }
            ],
            model="llama3-70b-8192",
        )
        return chat_completion.choices[0].message.content

class Phi3(LLM):
    
    def __init__(self):
        import ollama
        self.client = ollama

    def generate(self, system_prompt: str, user_prompt: str) -> str:
        response = self.client.chat(model="phi3", messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ])
        return response["message"]["content"]

class FactoryLLM:
    
    @staticmethod
    def create_model(model_name: str) -> LLM:
        """
        Create a model based on the name passed as a parameter
        
        Args:
            model_name: The name of the model to create
        
        Returns:
            The model created
        """
        class_llm = {
            "llama3": LLama3,
            "phi3": Phi3
        }
        
        if model_name not in class_llm:
            raise ValueError(f"model must be in {class_llm.keys()}")
        
        return class_llm[model_name]()

model_name = "phi3"
model = FactoryLLM.create_model(model_name)

print(model.generate("Hola", "Hola"))

¡Hola! ¿En qué puedo ayudarte hoy?


In [4]:
# Role play

prompt_role_play = {
    "system_prompt": "Imagina que eres un profesor de matemáticas y un estudiante te pregunta cómo resolver una ecuación de segundo grado",
    "user_prompt": "¿Cómo se resuelve una ecuación de segundo grado?"
}

print(model.generate(**prompt_role_play))

print(100 * "-")
prompt_role_play = {
    "system_prompt": "Eres Federico García Lorca, escribiras como él usando su mismo estilo y recursos literarios",
    "user_prompt": "Escribe un poema sobre prompt engineering"
}

print(model.generate(**prompt_role_play))

Para resolver una ecuación cuadrática, puedes seguir estos pasos:

1. Identifica la ecuación en su forma estándar, que es ax^2 + bx + c = 0, donde a, b y c son constantes y x representa las variables desconocidas.

2. Calcula el discriminante (Δ), que es igual a b² - 4ac. El discriminante determina la naturaleza de las raíces de la ecuación:
   - Si Δ > 0, hay dos soluciones reales y distintas.
   - Si Δ = 0, hay una única solución real (raíz doble).
   - Si Δ < 0, no hay soluciones reales; en cambio, habrá dos soluciones complejas conjugadas.

3. Aplica la fórmula general para resolver ecuaciones cuadráticas: x = (-b ± √Δ) / (2a). Aquí, los valores de "+" y "-" indican las dos soluciones posibles del problema.

4. Simplifica la expresión resultante para obtener las respuestas exactas a las variables desconocidas x.

Recuerda siempre verificar tus soluciones introduciendo los valores de x en la ecuación original y asegurándote de que ambas partes del lado izquierdo sean iguales al valo

In [5]:
# Zero-shot vs. few-shot

prompt_zero_shot = {
    "system_prompt": "Evalúa el sentimiento del texto que se te pase",
    "user_prompt": "No estoy seguro de cómo me siento."
}

prompt_zero_shot_improved = {
    "system_prompt": "Evalúa el sentimiento del texto que se te pase, "
              "Solo puedes responder con: Positivo, Negativo o Neutral"
              "La salida debe ser Sentimiento: <Positivo/Negativo/Neutral>",
    "user_prompt": "No estoy seguro de cómo me siento."
}

prompt_few_shot = {
    "system_prompt": "Texto: Estoy muy feliz hoy, Sentimiento: Positivo"
              "Texto: Odio tener que levantarme temprano, Sentimiento: Negativo"
              "Texto: No me importa si llueve Sentimiento: Neutral"
              "Evalúa el sentimiento del texto que se te pase usando los ejemplos anteriores",
    "user_prompt": "Texto: No estoy seguro de cómo me siento."
}

print(model.generate(**prompt_zero_shot))
print(100*"-")
print(model.generate(**prompt_zero_shot_improved))
print(100*"-")
print(model.generate(**prompt_few_shot))

El sentimiento expresado en "No estoy seguro de cómo me siento." parece indicar ambigüedad o incertidumbre emocional. El individuo no está claro sobre sus propios sentimientos, lo que sugiere un estado de confusión o posible tensión interna. Este lenguaje refleja una sensación de desconcierto y puede ser interpretado como neutral o con tendencia a la duda emocional.
Sentimiento: Neutral
El texto "No estoy seguro de cómo me siento" transmite un sentimiento ambivalente o confuso, lo que es indicativo de un estado emocional neutro pero con tendencia hacia la incertidumbre o la duda. En comparación con los ejemplos anteriores, este texto no asigna claramente una actitud positiva o negativa como en el caso de "Estoy muy feliz hoy" (positivo) y "Odio tener que levantarme temprano" (negativo). En lugar de eso, la expresión de incertidumbre sugiere un estado emocional entre las líneas o sin una inclinación hacia el lado positivo o negativo. Este tipo de sentimiento puede ser relevante en situa

In [6]:
# Chain of thoughts

prompt_chain_of_thoughts = {
    "system_prompt": "Dado el problema que se te dé, debes resolverlo paso a paso y después dar una solución",
    "user_prompt": "¿Es 11436181919023821 un número primo?"
}

print(model.generate(**prompt_chain_of_thoughts))

Para determinar si el número 11436181919023821 es primo, debemos probarlo utilizando una prueba de primalidad. Sin embargo, debido a su gran tamaño, esta no sería práctica manualmente. En cambio, podemos emplear programación y herramientas especializadas para realizarlo. A continuación, se muestra un enfoque computacional:


1. Utilizar una librería de números primos o algoritmo como el Miller-Rabin para verificar la primalidad.

2. Implementar dicha biblioteca o algoritmo en un lenguaje de programación y ejecutarlo con el número dado.

3. Interpretar el resultado proporcionado por el programa.


Si utilizamos una implementación del Miller-Rabin, nos encontraríamos con que 11436181919023821 no es primo (para este ejemplo temporal), aunque siempre se debe confirmar la primalidad de manera exhaustiva usando un método completo.
suponiendo que el resultado del Miller-Rabin indique que no es primo, entonces:

La solución es que 11436181919023821 no es un número primo según la prueba de Mill

In [7]:
import random

# Meta-Prompt Engineering

prompt_meta_prompt = {
    "system_prompt": "Tu misión es generar prompts que sean puedas resolver una tarea específica, se te dará el resultado de la anterior iteración para que puedas mejorar tu respuesta.",
    "user_prompt": "Determina si el número dado es primo. Anterior salida: {previous_output}"
}

previous_output = ""

for i in range(5):
    
    system_prompt = model.generate(**{
        "system_prompt": prompt_meta_prompt["system_prompt"],
        "user_prompt": prompt_meta_prompt["user_prompt"].format(previous_output=previous_output)
    })
    
    print(system_prompt)
    print(100*"-")
    
    number = random.randint(1, 1000000)
    prompt = {
        "system_prompt": system_prompt,
        "user_prompt": f"¿Es {number} un número primo?"
    }
    
    previous_output = model.generate(**prompt)

Para determinar si un número dado, digamos x, es primo o no, debemos verificar si existe algún divisor entre 2 y la raíz cuadrada de x (donde la raíz cuadrada se calcula con una precisión adecuada). Si encontramos al menos un divisor en ese rango, el número no es primo. Aquí hay un prompt mejorado para evaluar si un número dado es primo:

"Dada la entrada del número x, analiza los posibles divisores entre 2 y la raíz cuadrada de x (usando una precisión adecuada al calcular la raíz). Si se encuentra algún divisor en ese rango, concluye que el número x no es primo. Si no hay divisores encontrados en ese rango, determina que el número x es primo."

Ejemplo de entrada: 29

Procesamiento: 
1. Calculamos la raíz cuadrada de 29 con precisión adecuada (en este caso sería aproximadamente 5.385164807).
2. Verificamos si 29 es divisible por cualquier número entre 2 y nuestra aproximación de su raíz cuadrada (incluyendo el propio número en ese rango, aunque no se recomienda utilizarlo debido a la 